In [13]:
import re
import string
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


ModuleNotFoundError: No module named 'torch'

In [4]:
df = pd.read_csv('prabowo_menjawab.csv')
df.head()

,Nama Akun,Komentar,Sumber
0,@azzahrasyamilarahmat5028,11/100 Is Real...,Narasi Newsroom
1,@faritrohman3969,Saya kecewa cuma gara gara beberapa anak kelua...,Narasi Newsroom
2,@moradepadli5676,Prabowo aneh,Narasi Newsroom
3,@anggaviali1188,Nonton tanpa baca komen itu lebih baik,Narasi Newsroom
4,@derikoswaranda3500,woyyy mana ini?? ko komenan ga ada yg pro pa w...,Narasi Newsroom


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15020 entries, 0 to 15019
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Nama Akun  15017 non-null  object
 1   Komentar   15016 non-null  object
 2   Sumber     15020 non-null  object
dtypes: object(3)
memory usage: 352.2+ KB


Normalisasi + Cleaning

In [5]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Load kamus slang eksternal (misalnya dari nasalsabila/kamus-alay)
slang_df = pd.read_csv('https://raw.githubusercontent.com/nasalsabila/kamus-alay/refs/heads/master/colloquial-indonesian-lexicon.csv')  # Unduh dari GitHub atau buat sendiri
slang_dict = dict(zip(slang_df['slang'], slang_df['formal']))

def bersihkan_komentar(text):
    # Lowercase
    text = text.lower()
    # Hapus URL
    text = re.sub(r'http\S+|www.\S+', '', text)
    # Hapus mention & hashtag
    text = re.sub(r'@\w+|#\w+', '', text)
    # Hapus angka & simbol non-alphabetic
    text = re.sub(r'[^a-z\s]', '', text)
    # Tokenisasi
    words = text.split()
    # Ganti kata slang
    words = [slang_dict.get(word, word) for word in words]
    # Stemming
    stemmed = [stemmer.stem(word) for word in words]
    return ' '.join(stemmed)


In [ ]:
df['Komentar_Bersih'] = df['Komentar'].astype(str).apply(bersihkan_komentar)
df.head()

In [12]:
# Load model IndoBERT sentiment
model_name = "w11wo/indonesian-sentiment-twitter"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Deteksi sentimen
labels = ['negative', 'neutral', 'positive']
hasil_prediksi = []

NameError: name 'AutoTokenizer' is not defined